In [44]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

In [45]:
url = "https://www.eversports.de/s/poda-studio"

In [46]:
response = requests.get(url)

# if response.status_code != 200:
#     print("Fehler beim Abrufen der Webseite")
#     return

polestudio_soup = BeautifulSoup(response.content, 'html.parser')

In [47]:
overviewbuttons = polestudio_soup.find_all('div', class_="MuiStack-root css-sgccrm")

button_text = []

for item in overviewbuttons:
    # Find all <a> elements within the current <div>
    a_elements = item.find_all('a')
    
    # Iterate through <a> elements and append their text content to the list
    for a in a_elements:
        text = a.text
        button_text.append(text)

button_text

['Übersicht', 'Klassen', 'Workshops', 'Videos', 'Preise', 'Team']

In [48]:
# Pole Studio Name
pole_studio_name = polestudio_soup.find('h1', class_='MuiTypography-root MuiTypography-h1 css-qinhw0').text

pole_studio_name

'poda Studio'

In [49]:
# Owner information
polestudio_contact_info = polestudio_soup.find_all('div', class_='css-1x2phcg')

contact_info = []

for div in polestudio_contact_info:
    a_tags = div.find_all('a')
    
    contact_row = {"E-Mail": None, "Homepage": None, "Telefon": None}
    
    for a in a_tags:
        href = a.get('href')
        
        if href:
            if href.startswith('mailto:'):
                contact_row["E-Mail"] = href.replace('mailto:', '')
            elif href.startswith('tel:'):
                contact_row["Telefon"] = href.replace('tel:', '')
            else:
                contact_row["Homepage"] = href

    contact_info.append(contact_row)
    
contact_row

{'E-Mail': ' info@podastudio.de',
 'Homepage': 'https://www.podastudio.de/',
 'Telefon': '+49241 95719917'}

In [50]:
# Address
address_element = polestudio_soup.find('p', class_='MuiTypography-root MuiTypography-body1 css-1619old')

address_text_list = []

if address_element:
    address_text = address_element.text

    # Split the address into its components
    address = address_text.split(',')
    street = address_text.split(',')[0]
    postal_code = address[1].split(" ")[1]
    town = address[1].split(" ")[2]  # Town is the first part

    # Append the components to the address_text_list
    address_text_list.append(town)
    address_text_list.append(address)
    address_text_list.append(postal_code)
    address_text_list.append(street)

    print("Address:", address)
    print("Town:", town)
    print("Postal Code:", postal_code)
    print("Street:", street)
else:
    print("Address element not found or doesn't have the specified class.")

Address: ['Martinstraße 10-12 ', ' 52062 Aachen']
Town: Aachen
Postal Code: 52062
Street: Martinstraße 10-12 


In [51]:
# Pole Studio Description 
description_text = polestudio_soup.find('p', class_='MuiTypography-root MuiTypography-body1 css-154jcan').text

description_text

'Flieg mit uns! Poda ist das größte Pole Dance & Luftakrobatik Studio in Aachen und der Euregio. Neben Pole Dance / Pole Sport Kursen bieten wir auch Luftakrobatik, Tanzkurse wie Chair Dance, High Heels & Floorwork sowie Fitness, Yoga und Flexibility Kurse ohne Stange und Online Kurse an. Ob Probestunden, 10er-Karten oder Abos, Privatstunden, Workshops, Pole Parties, Junggesellinnen - Abschiede oder Pole Shows - wir haben für jeden das richtige Angebot.'

In [52]:
# Rating
rating_element = polestudio_soup.find('p', class_='MuiTypography-root MuiTypography-body1 css-2g7rhg')

rating_text_list = []

if rating_element:
    rating_text = rating_element.text
    
    # Split the rating text based on the opening parenthesis and remove any trailing white spaces
    rating_parts = rating_text.split('(')
    overall_rating = rating_parts[0].strip()
    num_reviews = rating_parts[1].replace(')', '').strip()

    # Append the extracted parts to the list
    rating_text_list.append(overall_rating)
    rating_text_list.append(num_reviews)

    print("Overall Rating:", overall_rating)
    print("Number of Reviews:", num_reviews)
else:
    print("Rating element not found or doesn't have the specified class.")

Overall Rating: 4,9
Number of Reviews: 51 Rezensionen


In [53]:
# Rating Factors
items = polestudio_soup.find_all('div', class_='MuiStack-root css-95g4uk')

combined_items = []

# Iterate through the found elements and extract the desired information
for item in items:
    # Extract the text related to each item
    item_name = item.find('p', class_='MuiTypography-root MuiTypography-body1 css-1k55edk')
    item_score = item.find('p', class_='MuiTypography-root MuiTypography-body1 css-1y0caop')

    if item_name and item_score:
        combined_item = f"{item_name.text}: {item_score.text}"
        combined_items.append(combined_item)

# Print the combined list
print(combined_items)

['Raumqualität: 5.0', 'Freundlichkeit & Service: 4.9', 'Sauberkeit: 5.0', 'Atmosphäre: 5.0', 'Preis-Leistung: 4.9']


In [54]:
#Art (Sportarten und Aktivitäten)
art = polestudio_soup.find_all("p", class_ = "MuiTypography-root MuiTypography-body1 css-6ik050")

arten = []

for item in art:
    
    print(item.text)
    arten.append(item.text)

Fitness
Tanzen
Fitness
Yoga
Tanzen
Fitness
Tanzen
Fitness
51 Rezensionen
Fitness
Tanzen
Yoga
Poledance
Personal Training
Akrobatik
Power Pole
Stretching
Floorwork
Chairdance
Aerial Silk
Aerial Dance
Aerial Hoop
Aerial Akrobatik
Polesport
Online Klasse
Bodybalance
Workout
Dehnung/Flexibilität
Twerken
Luftartistik


In [55]:
owner = []
id = []

In [56]:
# create a dictionary from the given lists
pole_studio_overview = {
    'ID':id,
    'PoleStudio_Name': pole_studio_name,
    'Adresse': address,
    'PLZ': postal_code,
    'Stadt': town,
    'Straße': street,
    'Buttons': button_text,
    'Pole Studio Beschreibung': description_text,
    'Ratingscore': rating_text_list[0],
    'Reviewanzahl': rating_text_list[1],
    'Rating Faktoren': combined_items,
    'E-Mail': contact_row['E-Mail'],
    'Homepage': contact_row['Homepage'],
    'Telefon': contact_row['Telefon'],
    'Eversports Pole Studio Seite': url,
    'Owner':owner,
    'Art':arten
    }

# create pandas dataframe
pole_studio_overview_df = pd.DataFrame([pole_studio_overview])

# Make a datastamp
pole_studio_overview_df['Created Date'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
pole_studio_overview_df['Updated Date'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [57]:
pole_studio_overview_df

,ID,PoleStudio_Name,Adresse,PLZ,Stadt,Straße,Buttons,Pole Studio Beschreibung,Ratingscore,Reviewanzahl,Rating Faktoren,E-Mail,Homepage,Telefon,Eversports Pole Studio Seite,Owner,Art,Created Date,Updated Date
0,[],poda Studio,"[Martinstraße 10-12 , 52062 Aachen]",52062,Aachen,Martinstraße 10-12,"[Übersicht, Klassen, Workshops, Videos, Preise...",Flieg mit uns! Poda ist das größte Pole Dance ...,"4,9",51 Rezensionen,"[Raumqualität: 5.0, Freundlichkeit & Service: ...",info@podastudio.de,https://www.podastudio.de/,+49241 95719917,https://www.eversports.de/s/poda-studio,[],"[Fitness, Tanzen, Fitness, Yoga, Tanzen, Fitne...",2023-12-17 02:10:06,2023-12-17 02:10:06


In [58]:
#Online Studios (Pole Studio)

In [59]:
#Bildergalerie
pictures = polestudio_soup.find_all(div, class_="MuiStack-root css-1a7wvso")

KeyboardInterrupt: 

In [ ]:
Angebot
sale = find(p, class_="MuiTypography-root MuiTypography-body1 css-153qxhx")